In [23]:
import pandas as pd
import numpy as np

import copy
import warnings
warnings.filterwarnings('ignore')

path = "./UNSW-NB15 - CSV Files/"

# 1. Combine Data and Preprocessing

## 1.1 Import Datasets

In [36]:
# Construct an empty DF for storing the processed data
df = pd.DataFrame()

# Read code book from system
codebook = pd.read_csv(path+'NUSW-NB15_features.csv', encoding='cp1252')
codebook = codebook.drop('No.',axis=1)
codebook

,Name,Type,Description
0,srcip,nominal,Source IP address
1,sport,integer,Source port number
2,dstip,nominal,Destination IP address
3,dsport,integer,Destination port number
4,proto,nominal,Transaction protocol
5,state,nominal,Indicates to the state and its dependent proto...
6,dur,Float,Record total duration
7,sbytes,Integer,Source to destination transaction bytes
8,dbytes,Integer,Destination to source transaction bytes
9,sttl,Integer,Source to destination time to live value


In [38]:
# Read each datasets from system and append to the empty df
for i in range(1,4):
    temp = pd.read_csv(path+'UNSW-NB15_%d.csv'%i, names = codebook['Name'] )
    df = df.append(temp)
    df = df.reset_index(drop = True)

## 1.2 Preprocessing

### 1.2.1 attack_cat
- fill na as normal
- clean attack labels 

In [66]:
# During the prelimilary data exploration, 
# we found that there are some labels from attack_cat need to be gathered
# eg:("backdoor" and "backdoors"; " Fuzzers" & " Fuzzers " )
def attack_cat_processing(s):
    s = s.strip(' ')
    if s[-1]=="s" and s != 'Analysis':
        s = s[:-1]
    return s
    
# Fill normal attack cat 
df['attack_cat'] = df['attack_cat'].fillna('Normal')

# clean attack labels
df['attack_cat'] = df['attack_cat'].apply(lambda x: attack_cat_processing(x))

#### Needs to be noticed:
- ip(srcip & dstip), proto, and time service needs to recode
- Stime & Ltime?

- Extreme values needs to be clamp
- Balancing data 

In [70]:
print("Samples in total:", len(df))

# 0 for normal and 1 for attack records
print("Normal samples:", len(df[df.Label==0]))
print("Attack samples:", len(df[df.Label==1]))

Samples in total: 2100003
Normal samples: 1867614
Attack samples: 232389


In [68]:
# Data Desciption
df.describe(include='all')

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
count,2100003,2100003.0,2100003,2100003,2100003,2100003,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,...,2100003.0,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2100003,2.100003e+06
unique,43,100325.0,47,127484,135,16,NaN,NaN,NaN,NaN,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
top,59.166.0.4,1043.0,149.171.126.2,53,tcp,FIN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,NaN
freq,170743,149468.0,170658,436149,1280203,1266257,NaN,NaN,NaN,NaN,...,1056339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1867614,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,6.751155e-01,4.451429e+03,3.841703e+04,5.864869e+01,...,NaN,8.723637e+00,8.492137e+00,6.029005e+00,6.504727e+00,4.142399e+00,3.210584e+00,6.096303e+00,NaN,1.106613e-01
std,NaN,NaN,NaN,NaN,NaN,NaN,1.522083e+01,5.436547e+04,1.654823e+05,7.068249e+01,...,NaN,1.035580e+01,1.032129e+01,7.790589e+00,7.868388e+00,8.022338e+00,5.840231e+00,1.067967e+01,NaN,3.137123e-01
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.060000e-03,2.640000e+02,1.780000e+02,3.100000e+01,...,NaN,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.792100e-02,1.540000e+03,2.260000e+03,3.100000e+01,...,NaN,5.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,NaN,0.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.234760e-01,3.390000e+03,1.525800e+04,3.100000e+01,...,NaN,1.000000e+01,9.000000e+00,6.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,NaN,0.000000e+00


# 2. Data Processing

## 2.1 Clamping

To reduce the skewness of some distributions, it is recommended to remove the extreme values. A common approach is to remove the features that have a maximum value more than ten times higher than the median value, and set them to the 95th percentile. However, if the 95th percentile is very close to the maximum value, it may indicate that the tail of the distribution contains valuable information, and should not be pruned.

It's important to note that this approach should only be used on features with a maximum value exceeding ten times the median. This avoids over-pruning of bimodal and small value distributions.

In [69]:
# Clamp extreme Values
df_numeric = df.select_dtypes(include=[np.number])
df_numeric.describe(include='all')

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,...,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
count,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,...,1.158342e+06,1.103536e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06
mean,6.751155e-01,4.451429e+03,3.841703e+04,5.864869e+01,3.104114e+01,5.373348e+00,1.718504e+01,3.067284e+07,2.555565e+06,3.477248e+01,...,2.091153e-01,3.380678e-02,8.723637e+00,8.492137e+00,6.029005e+00,6.504727e+00,4.142399e+00,3.210584e+00,6.096303e+00,1.106613e-01
std,1.522083e+01,5.436547e+04,1.654823e+05,7.068249e+01,4.090932e+01,2.184343e+01,5.806628e+01,1.089634e+08,4.279519e+06,7.679977e+01,...,7.839408e-01,1.842670e-01,1.035580e+01,1.032129e+01,7.790589e+00,7.868388e+00,8.022338e+00,5.840231e+00,1.067967e+01,3.137123e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.060000e-03,2.640000e+02,1.780000e+02,3.100000e+01,2.900000e+01,0.000000e+00,0.000000e+00,1.183662e+05,3.459092e+04,2.000000e+00,...,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
50%,1.792100e-02,1.540000e+03,2.260000e+03,3.100000e+01,2.900000e+01,3.000000e+00,4.000000e+00,5.719882e+05,6.183206e+05,1.400000e+01,...,0.000000e+00,0.000000e+00,5.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
75%,2.234760e-01,3.390000e+03,1.525800e+04,3.100000e+01,2.900000e+01,7.000000e+00,1.400000e+01,1.824163e+06,3.237420e+06,4.600000e+01,...,0.000000e+00,0.000000e+00,1.000000e+01,9.000000e+00,6.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00
max,8.786638e+03,1.435577e+07,1.465753e+07,2.550000e+02,2.540000e+02,5.319000e+03,5.507000e+03,5.988000e+09,1.287619e+08,1.064600e+04,...,3.600000e+01,4.000000e+00,6.700000e+01,6.700000e+01,6.700000e+01,6.700000e+01,6.700000e+01,6.000000e+01,6.700000e+01,1.000000e+00


In [75]:
# Clamping the data directly, inplace original 
for col in df_numeric.columns:
    if df_numeric[col].max()>10*df_numeric[col].median() and df_numeric[col].max()>10 :
        df[col] = np.where(df[col]<df[col].quantile(0.95), df[col], df[col].quantile(0.95))

In [73]:
df.describe(include='all')

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
count,2100003,2100003.0,2100003,2100003,2100003,2100003,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,...,2100003.0,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2100003,2.100003e+06
unique,43,100325.0,47,127484,135,16,NaN,NaN,NaN,NaN,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
top,59.166.0.4,1043.0,149.171.126.2,53,tcp,FIN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,NaN
freq,170743,149468.0,170658,436149,1280203,1266257,NaN,NaN,NaN,NaN,...,1056339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1867614,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.608372e-01,3.056101e+03,1.282540e+04,5.864869e+01,...,NaN,8.351984e+00,8.121860e+00,5.531559e+00,6.042557e+00,3.659807e+00,3.033895e+00,5.724219e+00,NaN,1.106613e-01
std,NaN,NaN,NaN,NaN,NaN,NaN,4.792901e-01,4.715772e+03,2.109689e+04,7.068249e+01,...,NaN,9.122398e+00,9.078506e+00,5.892815e+00,6.118841e+00,6.103987e+00,5.150554e+00,9.382710e+00,NaN,3.137123e-01
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.060000e-03,2.640000e+02,1.780000e+02,3.100000e+01,...,NaN,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.792100e-02,1.540000e+03,2.260000e+03,3.100000e+01,...,NaN,5.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,NaN,0.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.234760e-01,3.390000e+03,1.525800e+04,3.100000e+01,...,NaN,1.000000e+01,9.000000e+00,6.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,NaN,0.000000e+00


In [76]:
print("Samples in total:", len(df))

# 0 for normal and 1 for attack records
print("Normal samples:", len(df[df.Label==0]))
print("Attack samples:", len(df[df.Label==1]))

Samples in total: 2100003
Normal samples: 1867614
Attack samples: 232389


## 2.2 Log

In [77]:
df_numeric = df.select_dtypes(include=[np.number])
df_before = df_numeric.copy()
DEBUG = 0

for feature in df_numeric.columns:
    if df_numeric[feature].nunique()>50:
        if df_numeric[feature].min()==0:
            df[feature] = np.log(df[feature]+1)
        else:
            df[feature] = np.log(df[feature])

df_numeric = df.select_dtypes(include=[np.number])

In [78]:
df.describe(include='all')

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
count,2100003,2100003.0,2100003,2100003,2100003,2100003,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,...,2100003.0,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2.100003e+06,2100003,2.100003e+06
unique,43,100325.0,47,127484,135,16,NaN,NaN,NaN,NaN,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN
top,59.166.0.4,1043.0,149.171.126.2,53,tcp,FIN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,NaN
freq,170743,149468.0,170658,436149,1280203,1266257,NaN,NaN,NaN,NaN,...,1056339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1867614,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.787140e-01,6.950937e+00,6.731545e+00,5.864869e+01,...,NaN,8.351984e+00,8.121860e+00,5.531559e+00,6.042557e+00,3.659807e+00,3.033895e+00,5.724219e+00,NaN,1.106613e-01
std,NaN,NaN,NaN,NaN,NaN,NaN,3.013446e-01,1.577997e+00,3.577663e+00,7.068249e+01,...,NaN,9.122398e+00,9.078506e+00,5.892815e+00,6.118841e+00,6.103987e+00,5.150554e+00,9.382710e+00,NaN,3.137123e-01
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.059439e-03,5.579730e+00,5.187386e+00,3.100000e+01,...,NaN,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.776231e-02,7.340187e+00,7.723562e+00,3.100000e+01,...,NaN,5.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,NaN,0.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.016960e-01,8.128880e+00,9.632925e+00,3.100000e+01,...,NaN,1.000000e+01,9.000000e+00,6.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,4.000000e+00,NaN,0.000000e+00


## 2.3 Reducing cardinalities

We are going to reduce the cardinality of some features to 5 or 6 by selecting the top 5 occurring labels and setting the remainder to seldom used '-' labels. This prevents the encoding process from causing a large increase in dimensionality.

In [95]:
df_cat = df.select_dtypes(exclude=[np.number])
df_cat.describe(include='all')

,srcip,sport,dstip,dsport,proto,state,service,ct_ftp_cmd,attack_cat
count,2100003,2100003,2100003,2100003,2100003,2100003,2100003,2100003,2100003
unique,6,6,6,6,6,6,5,6,6
top,-,-,-,-,tcp,FIN,-,0,Normal
freq,1247592,1770611,1248296,1228064,1280203,1266257,1152172,1056339,1867614


In [80]:
for feature in df_cat.columns:
    if df_cat[feature].nunique()>6:
        df[feature] = np.where(df[feature].isin(df[feature].value_counts().head().index), df[feature], '-')

In [82]:
print("Samples in total:", len(df))

# 0 for normal and 1 for attack records
print("Normal samples:", len(df[df.Label==0]))
print("Attack samples:", len(df[df.Label==1]))

Samples in total: 2100003
Normal samples: 1867614
Attack samples: 232389


## 2.? Balancing (optional)

In [94]:

from imblearn.combine import SMOTEENN

def balancing(x,y,target):
    '''
    balance the data
    
    input:  x -- the x values(dataframe)
            y -- the y values
            target -- the name of the target
            
    return: balanced dataframe
    '''    
    sme = SMOTEENN(random_state=42)
    x_new, Y_new = sme.fit_resample(x,y)
    
    x_new = pd.DataFrame(x_new,columns = x.columns.tolist() )
    Y_new = pd.DataFrame(Y_new,columns =target )
    temp_df = pd.concat([x_new,Y_new],axis = 1)
    return temp_df

ValueError: could not convert string to float: '59.166.0.0'